# Resume Classification Project

Loading necessary libraries

In [1]:
# Loading necessary libraries

!pip install transformers datasets pandas

In [2]:
!pip install opendatasets

In [3]:
# Loading Kaggle datsets:

import pandas as pd
from datasets import load_dataset
import opendatasets as od

dataset = od.download(
    "https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset"
)

#dataset = datasets.load_dataset("csv", data_files="huggingface/snehaanbhawal/resume-dataset")

# Seperating features and lables

#features = dataset['train']["Resume_str"]
#labels = dataset['train']["Category"]

Skipping, found downloaded files in "./resume-dataset" (use force=True to force download)


In [4]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments

In [5]:
csv_path = "/content/resume-dataset/Resume/Resume.csv"
resume_df = pd.read_csv(csv_path)

In [6]:
!pip install PyPDF2

In [7]:
import PyPDF2

def extract_text_from_pdf(pdf_file):

  """
    Extract text from a PDF file.

    Args:
    pdf_file (file object): The PDF file object.

    Returns:
    str: The extracted text from the PDF.
    """
    # Create a PDF reader object
  pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Initialize an empty string to store the extracted text
  text = ""

    # Iterate through each page of the PDF
  for page_num in range(len(pdf_reader.pages)):

        # Extract text from the current page
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

  return text


In [9]:
# Loading the PDF files and extracted text
import os
resume_texts = []
pdf_root_folder = "/content/resume-dataset/data/data"

#pdf_folder = "/content/resume-dataset/data/data/"
for index, row in resume_df.iterrows():
    category_folder = os.path.join(pdf_root_folder, row['Category'])
    pdf_path = os.path.join(category_folder, f"{row['ID']}.pdf")
    with open(pdf_path, "rb") as f:
        text = extract_text_from_pdf(f)
        resume_texts.append(text)

In [10]:
# Adding the resume text to the dataframe

resume_df['Resume_text'] = resume_texts

In [11]:
# Define category mapping
category_mapping = {
    "HR": 0,
    "Designer": 1,
    "Information-Technology": 2,
    "Teacher": 3,
    "Advocate": 4,
    "Business-Development": 5,
    "Healthcare": 6,
    "Fitness": 7,
    "Agriculture": 8,
    "BPO": 9,
    "Sales": 10,
    "Consultant": 11,
    "Digital-Media": 12,
    "Automobile": 13,
    "Chef": 14,
    "Finance": 15,
    "Apparel": 16,
    "Engineering": 17,
    "Accountant": 18,
    "Construction": 19,
    "Public-Relations": 20,
    "Banking": 21,
    "Arts": 22,
    "Aviation": 23
}

In [12]:
# Map category names to category IDs

resume_df['Category_id'] = resume_df['Category'].map(category_mapping)

In [13]:
# Converting the dataframe to HuggingFace dataset format
from datasets import Dataset
dataset = Dataset.from_pandas(resume_df)

In [14]:
dataset

Dataset({
    features: ['ID', 'Resume_str', 'Resume_html', 'Category', 'Resume_text', 'Category_id'],
    num_rows: 2484
})

In [15]:
# Fine tunning a text classification model (using DistilBERT)

model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(category_mapping))
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Tokenize the dataset
def tokenize_function(examples):
  return tokenizer(examples['Resume_text'],padding='max_length',truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2484 [00:00<?, ? examples/s]

In [17]:
!pip install accelerate -U

In [18]:
!pip install transformers[torch]

In [19]:
# Defininf training arguments

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir="./logs",
    output_dir="./results",
    num_train_epochs=3,
)


In [ ]:
# Define the percentage of data for validation
validation_percentage = 0.2

# Calculate the number of samples for validation
num_validation_samples = int(len(train_dataset) * validation_percentage)

# Split the dataset into training and validation sets
train_dataset = Dataset({key: train_dataset[key][:-num_validation_samples] for key in train_dataset.keys()})
val_dataset = Dataset({key: train_dataset[key][-num_validation_samples:] for key in train_dataset.keys()})

# Verify the lengths of the datasets
print("Training Dataset Length:", len(train_dataset))
print("Validation Dataset Length:", len(val_dataset))

In [35]:
# Print a few samples from the training dataset
"""print("Training Dataset Samples:")
for i in range(min(5, len(train_dataset))):
    print(train_dataset[i])

# Print a few samples from the validation dataset
print("\nValidation Dataset Samples:")
for i in range(min(5, len(val_dataset))):
    print(val_dataset[i])"""

Training Dataset Samples:


KeyError: 0

In [36]:
# Print the first few elements of the training dataset
"""print("First few elements of the training dataset:")
for i, sample in enumerate(train_dataset):
    print(sample)
    if i == 4:  # Print the first 5 elements
        break"""

First few elements of the training dataset:
ID
Resume_str
Resume_html
Category
Resume_text


In [37]:
# Print the first few elements of the training dataset
"""print("First few elements of the training dataset:")
for i in range(min(5, len(train_dataset))):
    sample = {key: train_dataset[key][i] for key in train_dataset.keys()}
    print(sample)"""

First few elements of the training dataset:
{'ID': 16852973, 'Resume_str': "         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and

In [38]:
# Define the features of interest
"""features_of_interest = ['ID', 'Resume_str', 'Category']

# Print the first few elements of the training dataset with specific features
print("First few elements of the training dataset:")
for i in range(min(5, len(train_dataset))):
    sample = {feature: train_dataset[feature][i] for feature in features_of_interest}
    print(sample)"""

First few elements of the training dataset:
{'ID': 16852973, 'Resume_str': "         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and

In [28]:
from sklearn.model_selection import train_test_split

# Split dataset into training and validation sets
train_dataset, val_dataset = train_test_split(tokenized_dataset, test_size=0.2, random_state=50)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    output_dir="./results",
    num_train_epochs=3,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
# Define the percentage of data for validation
validation_percentage = 0.2

# Calculate the number of samples for validation
num_validation_samples = int(len(train_dataset) * validation_percentage)

# Split the dataset into training and validation sets
train_dataset = Dataset({key: train_dataset[key][:-num_validation_samples] for key in train_dataset.keys()})
val_dataset = Dataset({key: train_dataset[key][-num_validation_samples:] for key in train_dataset.keys()})

# Verify the lengths of the datasets
print("Training Dataset Length:", len(train_dataset))
print("Validation Dataset Length:", len(val_dataset))

In [47]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [43]:
#train_dataset

In [48]:
print("Train Dataset Length:", len(train_dataset))
print("Validation Dataset Length:", len(val_dataset))

Train Dataset Length: 8
Validation Dataset Length: 8


In [32]:
#print("Tokenized Dataset Length:", len(tokenized_dataset))


Tokenized Dataset Length: 2484


In [49]:
# Train the model
trainer.train()

KeyError: 6

In [27]:
# Evaluate the model
trainer.evaluate()


TypeError: vars() argument must have __dict__ attribute